<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#methods" data-toc-modified-id="methods-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>methods</a></span></li></ul></div>

## imports

In [4]:
import pandas as pd
import numpy as np
import pandas as pd
import mysql.connector
from tqdm import tqdm
import h5py
from sqlalchemy import types, create_engine 
import os
from os import listdir
from os.path import isfile, join
import config
import time

## methods

In [9]:
def get_files_list(): # gets list of filepath strings in ~/data
    mypath = os.getcwd()+'/data/'
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for f in files:
        f = mypath+f # append full path for h5py loading
    files = list(map(lambda f: mypath+f, files))
    return files 

def process_orbit(h5): # filename string. adds h5's observations to dataframe
    # the formatting/array shape is all uniform. thanks, NASA
    f = h5py.File(h5, 'r') # read file
    geo = f['GEOLOCATION_DATA'] # h5 Groups architecture is similar to dict
    sci = f['SCIENCE_DATA']
    # hdf.Datasets -> np.array -> flatten -> list. faster than looping through each matrix
    lat = list(geo['Latitude'].value.ravel())
    long = list(geo['Longitude'].value.ravel()) # is there a less verbose way to do this?
    sat_lat = list(geo['SpacecraftLatitude'].value.ravel())*36 # extend the 1d arrays
    sat_long = list(geo['SpacecraftLongitude'].value.ravel())*36 # don't forget to ravel like i did
    sat_alt = list(geo['SpacecraftAltitude'].value.ravel())*36
    time = list(geo['TimeUTC'].value.ravel())*36 # 36 measurements per position means one "time" value for every consecutive 36 measurements
    sza = list(geo['SolarZenithAngle'].value.ravel())
    pbl = list(sci['ColumnAmountSO2_PBL'].value.ravel())
    anom = list(sci['Flag_SAA'].value.ravel())
    volc = list(sci['Flag_SO2'].value.ravel())
    # combine lists into df
    new = pd.DataFrame(list(zip(lat, long, sat_lat, sat_long, sat_alt, time, sza, pbl, anom, volc)),
                       columns=["lat", "long", "sat_lat", "sat_long", 
                                'sat_alt', "time", "sza", "pbl", "anom", 'volc'])
    new['time'] = new['time'].astype(str) # change time format
    new['time'] = new['time'].apply(lambda st: st[2:12]+' '+st[13:21]) # change time format
    return new # returns new df

def make_engine():
    user = config.user # substitute your own username, password & SQL server host
    pw = config.pw
    host = config.host
    db = config.db
    connst = f'mysql+pymysql://{user}:{pw}@{host}/{db}'
    engine = create_engine(connst, echo=False) # don't set pool_recycle
    return engine

def process_h5s(files, engine): # filepath strings, sqlalchemy engine
    # loads all h5s into dataframes -> uploads to SQL
    # to keep track of progress
    files = get_files_list() # get filepath strings
    engine = 
    for f in tqdm(files): # this took 18 hours for 40,000 11k-length h5 files on my laptop
        df = process_orbit(f)
        df.to_sql('so2', con=engine, if_exists='append') # sqlalchemy takes care of sessions, commits etc rather nicely
    return 

def setup(): # get engine and filestring list
    print('establishing engine...')
    engine = make_engine() # establish connection to server
    files = get_files_list() # get filepath strings
    return engine, files

In [10]:
engine, files = setup()
print(f'processing {len(files)} files...')
process_h5s(files, engine) # process each file & upload to server

establishing engine...




  0%|          | 0/38976 [00:00<?, ?it/s]

processing 38976 files...




  0%|          | 1/38976 [00:05<63:04:45,  5.83s/it]

  0%|          | 2/38976 [00:07<49:37:27,  4.58s/it]

  0%|          | 3/38976 [00:09<40:18:15,  3.72s/it]

KeyboardInterrupt: 